In [2]:
from langdetect import detect
import os
import re
from collections import defaultdict

In [3]:
import pandas as pd

data = defaultdict(lambda: {'topic': '', 'content': '', 'messages': []})

for file_path in os.listdir("cleaned_intercom_messages"):
    csv = pd.read_csv(f"cleaned_intercom_messages/{file_path}")
    csv = csv[csv['clean_body'].notna()]
    bodies = csv["clean_body"]
    ids = zip(csv["key_intercomconversation"], csv["key_iteration"])
    for id, body in zip(ids, bodies):
        m = re.search(" Selected category: (.+) Content: (.+) uid:", body)
        if m is not None:
            data[id]['topic'] = m.group(1)
            data[id]['content'] = m.group(2)
        else:
            data[id]['messages'].append(body)

df = pd.DataFrame.from_dict(data, orient="index")
print(len(df[df["topic"] == '']))
print(len(df[df["topic"] != '']))

329564
10


In [4]:
import re

In [5]:
messages = {}

#for index, row in test_df.iterrows():
for index, row in df.iterrows():
    msg = re.sub(r'{{[^}]*}}', '', " ".join(row['messages']))
    messages[index] = msg

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [9]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ingma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
stop_words.extend(["wo", "n't", "'m", "ca", "'ll", "'re", "'ve", "'d"])

In [34]:
#words = word_tokenize(messages[13850823877])
#words = messages[13850823877].split(" ")
lemmatizer = WordNetLemmatizer()

lemmas = {}
stop_words = set(stop_words)
length = len(messages)
for i, (key, value) in enumerate(messages.items()):
    words = word_tokenize(value)
    lem = []
    if i % (length // 100) == 0:
        print(i, length)
    for word in words:
        lemma = lemmatizer.lemmatize(word)
        if len(lemma) > 1:
            lem.append(lemma.lower())
    lem = [w for w in lem if not w in stop_words]
    lemmas[key] = lem
    
#print(lemmas)

0 329574
3295 329574
6590 329574
9885 329574
13180 329574
16475 329574
19770 329574
23065 329574
26360 329574
29655 329574
32950 329574
36245 329574
39540 329574
42835 329574
46130 329574
49425 329574
52720 329574
56015 329574
59310 329574
62605 329574
65900 329574
69195 329574
72490 329574
75785 329574
79080 329574
82375 329574
85670 329574
88965 329574
92260 329574
95555 329574
98850 329574
102145 329574
105440 329574
108735 329574
112030 329574
115325 329574
118620 329574
121915 329574
125210 329574
128505 329574
131800 329574
135095 329574
138390 329574
141685 329574
144980 329574
148275 329574
151570 329574
154865 329574
158160 329574
161455 329574
164750 329574
168045 329574
171340 329574
174635 329574
177930 329574
181225 329574
184520 329574
187815 329574
191110 329574
194405 329574
197700 329574
200995 329574
204290 329574
207585 329574
210880 329574
214175 329574
217470 329574
220765 329574
224060 329574
227355 329574
230650 329574
233945 329574
237240 329574
240535 329574
24

In [35]:
all_lems = []

for key in lemmas:
    all_lems.extend(lemmas[key])

In [36]:
from collections import Counter

counter = Counter(all_lems).most_common()

In [37]:
stop_words_from_freq = ["pipedrive", "please", "help", "know", "let", "see", "'s", "need", "would", "question", "``", "day", "get", "one", "possible", "feel", "wa", "''", "ha", "want", "like", "thank", "already", "yes", "ok", "done", "still", "great", "could", "soon", "also", "anything", "well", "right", "sure", "um", "else"]

In [38]:
stop_words_from_freq = set(stop_words_from_freq)

In [39]:
new_lemmas = dict()

for key in lemmas:
    new_lemmas["_".join(map(str, key))] = [word for word in lemmas[key] if word not in stop_words_from_freq]

In [40]:
import json

with open("new_lems.json", "w", encoding="UTF-8") as f:
    f.write(json.dumps(new_lemmas))